## Data Preprocessing    

In [1]:
#importing neccessary libraries
import pandas as pd

In [2]:
#reading the csv file

df=pd.read_csv('lending_club_loans.csv')
df.shape

C:\Users\patel\AppData\Local\Temp\ipykernel_8872\4211197254.py:3: DtypeWarning: Columns (0,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('lending_club_loans.csv')


(42538, 115)

In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
#Feature Engineering

# Define the required columns
selected_columns = [
    'loan_amnt','installment' ,'emp_length', 'term', 'int_rate', 'home_ownership',
    'annual_inc', 'purpose', 'dti', 'fico_range_high',
    'funded_amnt', 'loan_status'
]


new_df = df[selected_columns]
new_df.shape

(42538, 12)

In [5]:
# Managing the loan_status
new_df['loan_status'] = new_df['loan_status'].replace("Charged Off", "Default")
new_df['loan_status']

# Keep only Fully Paid or Charged Off loan statuses
new_df = new_df[new_df['loan_status'].isin(['Fully Paid', 'Default'])]
new_df = new_df[new_df['home_ownership'] != "NONE"]

C:\Users\patel\AppData\Local\Temp\ipykernel_8872\158823421.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['loan_status'] = new_df['loan_status'].replace("Charged Off", "Default")


In [6]:
print(new_df['loan_status'].value_counts())

loan_status
Fully Paid    33583
Default        5654
Name: count, dtype: int64


In [7]:
print("Checking for the missing values")
print(new_df.isna().sum())

Checking for the missing values
loan_amnt             0
installment           0
emp_length         1057
term                  0
int_rate              0
home_ownership        0
annual_inc            0
purpose               0
dti                   0
fico_range_high       0
funded_amnt           0
loan_status           0
dtype: int64


In [8]:
#The emp_length column has categorical values the below function will encode it

def encode_emp_length(emp):
    if emp=='< 1 year':
        return 0
    elif emp=='10+ years':
        return 10
    else:
        return int(emp.split()[0]) if isinstance(emp,str) and emp[0].isdigit else None

In [9]:
# runing the ffunction on the emp_length column
new_df['emp_length']=new_df['emp_length'].apply(encode_emp_length)

In [10]:
new_df[new_df["term"].isna()]

,loan_amnt,installment,emp_length,term,int_rate,home_ownership,annual_inc,purpose,dti,fico_range_high,funded_amnt,loan_status


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

def fill_missing_values_nn(df):
    # Separate the rows with and without missing 'emp_length'
    df_not_na = df[df['emp_length'].notna()]
    df_na = df[df['emp_length'].isna()]

    # Features (X) and target (y) for rows where 'emp_length' is not missing
    X = df_not_na.drop(columns=['emp_length'])
    y = df_not_na['emp_length']

    # Label Encoding for categorical columns
    categorical_columns = X.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        label_encoder = LabelEncoder()
        X[col] = label_encoder.fit_transform(X[col])

    # Train-Test Split for model evaluation
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train Multi-Layer Perceptron (MLP) Classifier
    model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
    model.fit(X_train, y_train)

    # Predict missing values in the 'emp_length' column
    X_na = df_na.drop(columns=['emp_length'])  # Features for rows with missing 'emp_length'

    # Apply the same Label Encoding to the categorical columns in the missing data
    for col in categorical_columns:
        label_encoder = LabelEncoder()
        X_na[col] = label_encoder.fit_transform(X_na[col])

    # Predict the missing 'emp_length' values using the trained model
    df.loc[df['emp_length'].isna(), 'emp_length'] = model.predict(X_na)

    # Optional: Evaluate the model's performance on the test set
    print("Model performance on test set:")
    print(f"Accuracy Score: {accuracy_score(y_test, model.predict(X_test))}")
    print("Classification Report:")
    print(classification_report(y_test, model.predict(X_test)))

    return df

# Example usage:
new_df = fill_missing_values_nn(new_df)

# Verify that the missing values are filled
print(new_df.isna().sum())


Model performance on test set:
Accuracy Score: 0.11563645887899424
Classification Report:
              precision    recall  f1-score   support

         0.0       0.12      0.97      0.21       891
         1.0       0.00      0.00      0.00       664
         2.0       0.09      0.01      0.02       887
         3.0       0.00      0.00      0.00       794
         4.0       0.50      0.00      0.00       697
         5.0       0.07      0.02      0.03       637
         6.0       0.00      0.00      0.00       404
         7.0       0.00      0.00      0.00       390
         8.0       0.00      0.00      0.00       285
         9.0       0.00      0.00      0.00       264
        10.0       0.00      0.00      0.00      1723

    accuracy                           0.12      7636
   macro avg       0.07      0.09      0.02      7636
weighted avg       0.08      0.12      0.03      7636

loan_amnt          0
installment        0
emp_length         0
term               0
int_rate     

c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
new_df.head()

,loan_amnt,installment,emp_length,term,int_rate,home_ownership,annual_inc,purpose,dti,fico_range_high,funded_amnt,loan_status
0,5000.0,162.87,10.0,36 months,10.65%,RENT,24000.0,credit_card,27.65,739.0,5000.0,Fully Paid
1,2500.0,59.83,0.0,60 months,15.27%,RENT,30000.0,car,1.00,744.0,2500.0,Default
2,2400.0,84.33,10.0,36 months,15.96%,RENT,12252.0,small_business,8.72,739.0,2400.0,Fully Paid
3,10000.0,339.31,10.0,36 months,13.49%,RENT,49200.0,other,20.00,694.0,10000.0,Fully Paid
5,5000.0,156.46,3.0,36 months,7.90%,RENT,36000.0,wedding,11.20,734.0,5000.0,Fully Paid


In [13]:
#the int_rate has a % sign which is not needed so the below function will remove the sign and will clean it
def clean_int_rate(int_rate):
    return float(int_rate[:-1])
new_df['int_rate']=new_df['int_rate'].apply(clean_int_rate)
new_df

,loan_amnt,installment,emp_length,term,int_rate,home_ownership,annual_inc,purpose,dti,fico_range_high,funded_amnt,loan_status
0,5000.0,162.87,10.0,36 months,10.65,RENT,24000.0,credit_card,27.65,739.0,5000.0,Fully Paid
1,2500.0,59.83,0.0,60 months,15.27,RENT,30000.0,car,1.00,744.0,2500.0,Default
2,2400.0,84.33,10.0,36 months,15.96,RENT,12252.0,small_business,8.72,739.0,2400.0,Fully Paid
3,10000.0,339.31,10.0,36 months,13.49,RENT,49200.0,other,20.00,694.0,10000.0,Fully Paid
5,5000.0,156.46,3.0,36 months,7.90,RENT,36000.0,wedding,11.20,734.0,5000.0,Fully Paid
...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,78.42,4.0,36 months,8.07,MORTGAGE,110000.0,home_improvement,11.33,764.0,2500.0,Fully Paid
39782,8500.0,275.38,3.0,36 months,10.28,RENT,18000.0,credit_card,6.40,694.0,8500.0,Fully Paid
39783,5000.0,156.84,0.0,36 months,8.07,MORTGAGE,100000.0,debt_consolidation,2.30,744.0,5000.0,Fully Paid
39784,5000.0,155.38,0.0,36 months,7.43,MORTGAGE,200000.0,other,3.72,814.0,5000.0,Fully Paid


In [14]:
# new_df['purpose'].value_counts()

# Defining the proper Purpose values
def format_purpose(purpose):
    if '_' in purpose:
        return ' '.join([word.capitalize() for word in purpose.split('_')])
    else:
        return purpose.capitalize()

In [15]:
new_df['purpose']=new_df['purpose'].apply(format_purpose)
new_df

,loan_amnt,installment,emp_length,term,int_rate,home_ownership,annual_inc,purpose,dti,fico_range_high,funded_amnt,loan_status
0,5000.0,162.87,10.0,36 months,10.65,RENT,24000.0,Credit Card,27.65,739.0,5000.0,Fully Paid
1,2500.0,59.83,0.0,60 months,15.27,RENT,30000.0,Car,1.00,744.0,2500.0,Default
2,2400.0,84.33,10.0,36 months,15.96,RENT,12252.0,Small Business,8.72,739.0,2400.0,Fully Paid
3,10000.0,339.31,10.0,36 months,13.49,RENT,49200.0,Other,20.00,694.0,10000.0,Fully Paid
5,5000.0,156.46,3.0,36 months,7.90,RENT,36000.0,Wedding,11.20,734.0,5000.0,Fully Paid
...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,78.42,4.0,36 months,8.07,MORTGAGE,110000.0,Home Improvement,11.33,764.0,2500.0,Fully Paid
39782,8500.0,275.38,3.0,36 months,10.28,RENT,18000.0,Credit Card,6.40,694.0,8500.0,Fully Paid
39783,5000.0,156.84,0.0,36 months,8.07,MORTGAGE,100000.0,Debt Consolidation,2.30,744.0,5000.0,Fully Paid
39784,5000.0,155.38,0.0,36 months,7.43,MORTGAGE,200000.0,Other,3.72,814.0,5000.0,Fully Paid


In [16]:
#creating a new column named fico_category to categorise the FICO scores in the dataset
def assign_fico_category(fico_score):
    score_bins = [300, 579, 669, 739, 799, 850]
    score_labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
    
    for i in range(len(score_bins)-1):
        if fico_score >= score_bins[i] and fico_score <= score_bins[i+1]:
            return score_labels[i]
    
    return None  # In case of any unexpected values

new_df['fico_category'] = new_df['fico_range_high'].apply(assign_fico_category)



In [17]:

print(new_df[['fico_range_high', 'fico_category']].head())  # Verify the changes

   fico_range_high fico_category
0            739.0          Good
1            744.0     Very Good
2            739.0          Good
3            694.0          Good
5            734.0          Good


In [18]:
# Perform the calculations for monthly income and the installment to payment ratio
new_df['monthly_inc'] = round(new_df['annual_inc'] / 12,2)
new_df['installment_to_income_ratio_%'] = round((new_df['installment'] / new_df['monthly_inc']) * 100,2)

# Verify the changes
print(new_df[['monthly_inc', 'installment_to_income_ratio_%']].head())

   monthly_inc  installment_to_income_ratio_%
0       2000.0                           8.14
1       2500.0                           2.39
2       1021.0                           8.26
3       4100.0                           8.28
5       3000.0                           5.22


In [19]:
new_df.head(10)

,loan_amnt,installment,emp_length,term,int_rate,home_ownership,annual_inc,purpose,dti,fico_range_high,funded_amnt,loan_status,fico_category,monthly_inc,installment_to_income_ratio_%
0,5000.0,162.87,10.0,36 months,10.65,RENT,24000.0,Credit Card,27.65,739.0,5000.0,Fully Paid,Good,2000.00,8.14
1,2500.0,59.83,0.0,60 months,15.27,RENT,30000.0,Car,1.00,744.0,2500.0,Default,Very Good,2500.00,2.39
2,2400.0,84.33,10.0,36 months,15.96,RENT,12252.0,Small Business,8.72,739.0,2400.0,Fully Paid,Good,1021.00,8.26
3,10000.0,339.31,10.0,36 months,13.49,RENT,49200.0,Other,20.00,694.0,10000.0,Fully Paid,Good,4100.00,8.28
5,5000.0,156.46,3.0,36 months,7.90,RENT,36000.0,Wedding,11.20,734.0,5000.0,Fully Paid,Good,3000.00,5.22
6,7000.0,170.08,8.0,60 months,15.96,RENT,47004.0,Debt Consolidation,23.51,694.0,7000.0,Fully Paid,Good,3917.00,4.34
7,3000.0,109.43,9.0,36 months,18.64,RENT,48000.0,Car,5.35,664.0,3000.0,Fully Paid,Fair,4000.00,2.74
8,5600.0,152.39,4.0,60 months,21.28,OWN,40000.0,Small Business,5.55,679.0,5600.0,Default,Good,3333.33,4.57
9,5375.0,121.45,0.0,60 months,12.69,RENT,15000.0,Other,18.08,729.0,5375.0,Default,Good,1250.00,9.72
10,6500.0,153.45,5.0,60 months,14.65,OWN,72000.0,Debt Consolidation,16.12,699.0,6500.0,Fully Paid,Good,6000.00,2.56


In [20]:
#Run to save the cleaned dataset
new_df.to_csv("cleaned_dataset.csv", index=False)